In [ ]:
# IMPORT PACKAGES FOR PROJECT

# !pip install wordcloud, pandas, matplotlib, PIL
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
book_list2021 = {1:{'Name':"The richest man in Babylon", 'Author':"George S. Clason", 'Pages':224}
                 , 2:{'Name':"The slight edge", 'Author':"Jeff Olson", 'Pages':280}
                 , 3:{'Name':"Intermittent fasting basics", 'Author':"Lindsay Boyers", 'Pages':219}
                 , 4:{'Name':"The one thing", 'Author':"Gary Keller", 'Pages':240}
                 , 5:{'Name':"Who moved my cheese", 'Author':"Spencer Johnson", 'Pages':94}
                 , 6:{'Name':"Think and grow rich", 'Author':"Napoleon Hill", 'Pages':290}
                 , 7:{'Name':"Outwitting the devil: The secret to freedom and success", 'Author':"Napoleon Hill", 'Pages':302}
                 , 8:{'Name':"The four agreements", 'Author':"Don Miquel Ruiz", 'Pages':140}
                 , 9:{'Name':"Atomic habits", 'Author':"James Clear", 'Pages':298}
                 , 10:{'Name':"How to win friends and influence people", 'Author':"Dale Carnegie", 'Pages':250}
                 , 11:{'Name':"Trading in the zone", 'Author':"Mark Douglas", 'Pages':207}
                 , 12:{'Name':"Stone soup", 'Author':"Marcia Brown", 'Pages':52}
                 , 13:{'Name':"Rich dad's cashflow quadrant", 'Author':"Robert T. Kiyosaki", 'Pages':282}
                 , 14:{'Name':"A beginners guide to the stock market", 'Author':"Mathew R. Kratter", 'Pages':100}
                 , 15:{'Name':"Credit spread options for beginners", 'Author':"Freeman Publications", 'Pages':134}
                 , 16:{'Name':"Heart sick: Connecting the dots between hope, disappointment and healing", 'Author':"Conscious Core", 'Pages':107}
                 , 17:{'Name':"Thinking strategically: the competitive edge in business, politics and everyday life", 'Author':"Avinash K. Dixit & " "Barry J. Nalebuff", 'Pages':384}
                 , 18:{'Name':"Noise trading and illusory corrections in the US equity markets", 'Author':"Jennifer C. Bender & " "Carol L. Osler & " "David Simon", 'Pages':47}
                }

In [ ]:
BL21 = pd.DataFrame(book_list2021).transpose()
BL21

In [ ]:
pages_read21 = BL21['Pages'].sum()
print(f"Number of pages read in 2021: {format(pages_read21, ',d')}")
print(f"Average number of pages read per month in 2021: {round(pages_read21/12)}")
print(f"Average number of pages read per day in 2021: {round(pages_read21/365)}")

In [ ]:
word_string=""

for book in BL21['Name']:
     word_string += book.upper() + " "

word_cloud = WordCloud(
     background_color='black'
                       , scale=3, collocations=True
                       , colormap='tab20c'
                       , normalize_plurals=True
                       ).generate(word_string)

plt.figure(figsize = (20,6), facecolor = 'black')
plt.imshow(word_cloud, interpolation = 'bessel')
plt.axis('off')
plt.title("2021 Reading List", fontweight='bold', color='black', backgroundcolor='gold')

plt.show()

In [ ]:
book_list2022 = {1:{'Name':"The subtle art of not giving a F*ck", 'Author':"Mark Manson", 'Pages':206}
                 , 2:{'Name':"Why we sleep; unlocking the power of sleep and dreams", 'Author':"Mathew Walker", 'Pages':323}
                 , 3:{'Name':"Ingredients: the strange chemistry of what we put in and on us", 'Author':"Goerge Zaidan", 'Pages':299}
                 , 4:{'Name':"We should all be millionaires: A woman's guide to earning more, building wealth, and gaining economic power", 'Author':"Rachel Rodgers", 'Pages':276}
                 , 5:{'Name':"The five love languages", 'Author':"Gary Chapman", 'Pages':272}
                 , 6:{'Name':"The art of saying NO", 'Author':"Damon Zahariades", 'Pages':160}
                 , 7:{'Name':"If you had controlling parents: How to make peace with your past and take your place in the world", 'Author':"Dan Neuharth", 'Pages':274}
                 , 8:{'Name':"Abundance beyond trauma: Discovering the courage for change and commitment to yourself", 'Author':"Jeannine L. Rashidi", 'Pages':169}
                 , 9:{'Name':"Dear Girl", 'Author':"Amy Krouse Rosenthal & " "Paris Rosenthal", 'Pages':40}
                }

In [ ]:
BL22 = pd.DataFrame(book_list2022).transpose()
BL22

In [ ]:
pages_read22 = BL22['Pages'].sum()
print(f"Number of pages read in 2022: {format(pages_read22, ',d')}")
print(f"Average number of pages read per month in 2022: {round(pages_read22/12)}")
print(f"Average number of pages read per day in 2022: {round(pages_read22/365)}")

In [ ]:
word_string=""

for book in BL22['Name']:
     word_string += book.upper() + " "

word_cloud = WordCloud(background_color='black',scale=3, normalize_plurals = True).generate_from_text(word_string)

plt.figure(figsize = (20,6), facecolor='black')
plt.imshow(word_cloud, interpolation = 'catrom')
plt.axis('off')
plt.title("2022 Reading List", fontweight='bold', color='black', backgroundcolor='gold')

plt.show()

In [ ]:
book_list2023 = {"Name":
    ["The 7 habits of highly effective people"
     ,"Why your weirdness is wonderful: Embrace your quirks and live your strengths"]
                 , "Author":["Stephen R. Covey","Laurie Wallin"]
                 , "Pages":[556,193]
                 }

In [ ]:
BL23 = pd.DataFrame(book_list2023)
BL23.index = BL23.index + 1
BL23

In [ ]:
pages_read23 = BL23['Pages'].sum()
print(f"Number of pages read in 2023: {format(pages_read23, ',d')}")
print(f"Average number of pages read per month in 2023: {round(pages_read23/12)}")
print(f"Average number of pages read per day in 2023: {round(pages_read23/365)}")

In [ ]:
word_string=""

for book in BL23['Name']:
     word_string += book.upper() + " "

word_cloud = WordCloud(background_color='black',scale=3, normalize_plurals = True).generate_from_text(word_string)

plt.figure(figsize = (20,6), facecolor='black')
plt.imshow(word_cloud, interpolation = 'quadric')
# 'antialiased', 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman'
plt.axis('off')
plt.title("2023 Reading List", fontweight='bold', color='black', backgroundcolor='gold')

plt.show()

In [ ]:
BL21['Year Read'] = 2021
BL22['Year Read'] = 2022
BL23['Year Read'] = 2023

In [ ]:
comb_list = pd.concat([BL21, BL22, BL23]).sort_values(by='Name').set_index([pd.Index(range(1, len(BL21) + len(BL22) + len(BL23) + 1))])
comb_list

In [ ]:
word_string=""

for book in comb_list['Name']:
     word_string += book.upper() + " "

word_cloud = WordCloud(background_color='black',scale=3, collocations=True, colormap='tab20c', normalize_plurals=True).generate_from_text(word_string)

plt.figure(figsize = (20,10), facecolor=(0,0,0,0.49))
plt.imshow(word_cloud)
plt.axis('off')
plt.title("Combined Reading List", fontweight='bold', color='gold', backgroundcolor='black', fontsize=15)

plt.show()

In [ ]:
print('Key Words in Book Titles')
comb_list[comb_list['Name'].str.contains(
    'power| abundance| heal| you| habit| time| invest| change| grow| trauma'
    , case=False)]

In [ ]:
# Save file as CSV to create a power BI dashboard
labels = ['Short', 'Medium', 'Long']
bins = [0,200,280,1000]
comb_list['Bin'] = pd.cut(comb_list['Pages'], bins=bins, labels=labels)

# comb_list.to_csv('reading_list.csv')